In [1]:
from transformers import AutoTokenizer
import torch
import torch.nn
import torch.optim as optim

c:\Users\17539\anaconda3\envs\nlpasm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


先将大模型load进来

并且取出对应的词嵌入

```
Qwen2Config {
  "_name_or_path": "D:/models/Qwen2.5-7B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.45.2",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 152064
}
```

In [2]:
model_name = "D:/models/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
from qwen_source import  Qwen2ForCausalLM
model =  Qwen2ForCausalLM.from_pretrained(
    model_name,
    torch_dtype = 'auto',
    device_map = 'auto'
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 13.58it/s]
Some weights of Qwen2ForCausalLM were not initialized from the model checkpoint at D:/models/Qwen2.5-7B-Instruct and are newly initialized: ['model.vae.decoder_bn_layers.0.bias', 'model.vae.decoder_bn_layers.0.num_batches_tracked', 'model.vae.decoder_bn_layers.0.running_mean', 'model.vae.decoder_bn_layers.0.running_var', 'model.vae.decoder_bn_layers.0.weight', 'model.vae.decoder_bn_layers.1.bias', 'model.vae.decoder_bn_layers.1.num_batches_tracked', 'model.vae.decoder_bn_layers.1.running_mean', 'model.vae.decoder_bn_layers.1.running_var', 'model.vae.decoder_bn_layers.1.weight', 'model.vae.decoder_bn_layers.2.bias', 'model.vae.decoder_bn_layers.2.num_batches_tracked', 'model.vae.decoder_bn_layers.2.running_mean', 'model.vae.decoder_bn_layers.2.running_var', 'model.vae.decoder_bn_layers.2.weight', 'model.vae.decoder_bn_layers.3.bias', 'model.vae.decoder_bn_layers.3.num_batches_tracked', 'model.vae.decoder_bn_layers.

In [4]:
batch_size = 4
epochs = 10
lr = 5e-5
sequence_length = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
vae_optim_dict = []
qwen_optim_dict = []
for name, param in model.named_parameters():
    # split out the layer number
    name_l = name.split('.')
    layer_num = name_l[2] if name_l[1] == 'layers' else None
    if name_l[1] == "embed_tokens":
        param.requires_grad = False
    elif name_l[1] == "vae":
        vae_optim_dict.append({"params" : param, 'lr' : lr})
    
    if layer_num is not None:
        # frozen the lower layers to preserve the features
        # train the higher layers to fit into the specific downstream task
        layer_num = int(layer_num)
        if layer_num < 23:
            param.requires_grad = False
        elif layer_num >= 23:
            param.requires_grad = True
            qwen_optim_dict.append({"params" : param, 'lr' : lr})


In [6]:
qwen_optim_dict

[{'params': Parameter containing:
  tensor([[ 0.0073,  0.0101, -0.0306,  ...,  0.0028,  0.0239,  0.0106],
          [-0.0176, -0.0137, -0.0250,  ...,  0.0030,  0.0254,  0.0051],
          [-0.0236, -0.0111,  0.0029,  ..., -0.0104,  0.0015, -0.0266],
          ...,
          [-0.0084,  0.0072,  0.0063,  ...,  0.0117, -0.0132,  0.0099],
          [ 0.0349,  0.0206, -0.0105,  ..., -0.0013,  0.0186, -0.0165],
          [ 0.0112, -0.0164, -0.0049,  ..., -0.0074, -0.0020,  0.0082]],
         dtype=torch.bfloat16, requires_grad=True),
  'lr': 5e-05},
 {'params': Parameter containing:
  tensor([ 0.1582, -0.5000, -0.0292,  ..., -0.8125,  0.3203,  0.0640],
         dtype=torch.bfloat16, requires_grad=True),
  'lr': 5e-05},
 {'params': Parameter containing:
  tensor([[-0.0046,  0.0066,  0.0040,  ...,  0.0113,  0.0115, -0.0036],
          [ 0.0237, -0.0014,  0.0182,  ...,  0.0095, -0.0041, -0.0096],
          [-0.0044,  0.0003,  0.0153,  ..., -0.0053,  0.0097,  0.0131],
          ...,
          [ 

In [7]:
vae_optim = optim.AdamW(vae_optim_dict, lr=lr)
qwen_optim = optim.AdamW(qwen_optim_dict, lr = lr)

In [8]:
import glob
dataset_dir = 'D:/info/program/NLP/asm/dataset/'

train_list = glob.glob(dataset_dir + 'train*.json')
val_list = glob.glob(dataset_dir + 'val*.json')
len(train_list), len(val_list)

(10, 10)

In [9]:
from ruozhi_dataset import Ruozhi_dataset
from torch.utils.data import DataLoader

train1 = Ruozhi_dataset(train_list[0], tokenizer = tokenizer, sequence_length = sequence_length)
val1 = Ruozhi_dataset(val_list[0], tokenizer = tokenizer, sequence_length = sequence_length)

train_loader1 = DataLoader(train1, batch_size=batch_size, shuffle=True)
val_loader1 = DataLoader(val1, batch_size=len(val1), shuffle=True)

In [10]:
import datetime
print(datetime.datetime.now())

2024-11-01 16:46:15.008950


In [ ]:
model.to(device)
print("Start training")
for epoch in range(1):
    print("{} Epoch {} start".format(datetime.datetime.now(), epoch))
    i = 0
    for input_token, input_masking, output_token in train_loader1:
        i += 1
        input_token = input_token.squeeze(1)
        input_masking = input_masking.squeeze(1)
        output_token = output_token.squeeze(1)

        
        input_token = input_token.to(device)
        input_masking = input_masking.to(device)
        output_token = output_token.to(device)
        
        
        output = model(input_ids = input_token, 
                       attention_mask = input_masking,
                       labels = output_token)
        print("forward end")
        loss = output.loss
        vae_out = output.vae_out
        loss_dict = model.vae.loss_function(*vae_out, M_N=16 / 3200)
        vae_loss  = loss_dict["loss"]


        qwen_optim.zero_grad()
        loss.backward()
        qwen_optim.step()

        vae_optim.zero_grad()
        vae_loss.backward()
        vae_optim.step()
        if i % 10 == 0:
            print(f"{datetime.datetime.now()} Epoch {epoch + 1}, Training Loss: {loss.item()}")
    # for input_token, output_token in val_loader1:
    #     with torch.no_grad():
    #         input_token = input_token.view(batch_size, sequence_length)
    #         output_token = output_token.view(batch_size, sequence_length)

    #         input_token = input_token.to(device)
    #         output_token = output_token.to(device)


    #         output = model(input_ids = input_token, labels = output_token)
    #         loss = output.loss

    #         print(f"{datetime.datetime.now()} Epoch {epoch + 1}, Validation Loss: {loss.item()}")

Start training
2024-11-01 16:46:47.173572 Epoch 0 start
torch.Size([4, 128, 3584])


RuntimeError: could not create a primitive descriptor for an LSTM forward propagation primitive

In [13]:
input_token.shape

torch.Size([4, 128])

In [ ]:
for name, param in model.named_parameters():
    print(f"{name}    {param.requires_grad}")

model.embed_tokens.weight    False
model.vae.encoder_lstm_layers.0.weight_ih_l0    True
model.vae.encoder_lstm_layers.0.weight_hh_l0    True
model.vae.encoder_lstm_layers.0.bias_ih_l0    True
model.vae.encoder_lstm_layers.0.bias_hh_l0    True
model.vae.encoder_lstm_layers.0.weight_ih_l0_reverse    True
model.vae.encoder_lstm_layers.0.weight_hh_l0_reverse    True
model.vae.encoder_lstm_layers.0.bias_ih_l0_reverse    True
model.vae.encoder_lstm_layers.0.bias_hh_l0_reverse    True
model.vae.encoder_lstm_layers.0.weight_ih_l1    True
model.vae.encoder_lstm_layers.0.weight_hh_l1    True
model.vae.encoder_lstm_layers.0.bias_ih_l1    True
model.vae.encoder_lstm_layers.0.bias_hh_l1    True
model.vae.encoder_lstm_layers.0.weight_ih_l1_reverse    True
model.vae.encoder_lstm_layers.0.weight_hh_l1_reverse    True
model.vae.encoder_lstm_layers.0.bias_ih_l1_reverse    True
model.vae.encoder_lstm_layers.0.bias_hh_l1_reverse    True
model.vae.encoder_lstm_layers.1.weight_ih_l0    True
model.vae.encode